In [1]:
import os
from google import genai
from google.genai import types


# create client
api_key = os.getenv("GEMINI_API_KEY","xxx")
client = genai.Client(api_key=api_key)


# path to the file to upload
csv_file_path = "../assets/portfolio_transactions.csv" # Repalce with your own file path

# Upload the file to the File API
file = client.files.upload(file=csv_file_path)

In [2]:
prompt =  """Add a 'TotalValue' column. For 'BUY' actions, calculate it as (Quantity * PricePerShare) + Commission. For 'SELL' actions, calculate it as (Quantity * PricePerShare) - Commission. 
Return a helpful message and the return as a csv file."""

# Generate the response
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[file, prompt],
    config=types.GenerateContentConfig(
        tools=[types.Tool(code_execution=types.ToolCodeExecution)]
    ),
)

ClientError: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'The mime type: application/vnd.ms-excel is not supported for code execution.', 'status': 'INVALID_ARGUMENT'}}

In [ ]:
print(response.text)

In [ ]:
import pandas as pd
from IPython.display import Markdown

output_file = "response.csv"

for part in response.candidates[0].content.parts:
    if part.text:
        display(Markdown(part.text))
    if part.inline_data:
        with open(output_file, "wb") as f:
            f.write(part.inline_data.data)
        display(Markdown(f"CSV file saved to {output_file}"))
        display(Markdown(f"Preview of the first few rows of the CSV file:"))
        display(pd.read_csv(output_file).head())
        

In [ ]:
from IPython.display import Markdown

prompt =  """Calculate the total amount spent on 'BUY' transactions and the total amount received from 'SELL' transactions based on the 'TotalValue' column. Also, calculate the net cash flow (Total Received - Total Spent) and the total commission paid across all transactions. 
Return a helpful message and markdown table with the results."""

# Generate the response
response = client.models.generate_content(
    model="gemini-2.5-flash-preview-04-17",
    contents=[file, prompt],
    config={"tools": [{"code_execution": {}}]},
)

display(Markdown(response.text))